In [1]:
import pandas as pd
import us
from datetime import datetime
from warnings import simplefilter
from openpyxl import load_workbook
import psycopg2
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

# Load the data from CSV files with header in the first row


<h1>Extract only NY data and save as CSV</h1>

In [7]:
demo_datas = pd.read_csv('./IRS_Data_2021.csv', header=0, dtype={'zipcode': str})

In [8]:
ny_df = demo_datas[demo_datas['STATE'] == 'NY'].reset_index(drop=True)

In [10]:
ny_df.tail()

,STATEFIPS,STATE,zipcode,agi_stub,N1,mars1,MARS2,MARS4,ELF,CPREP,...,N85300,A85300,N11901,A11901,N11900,A11900,N11902,A11902,N12000,A12000
9211,36,NY,99999,2,17620.0,11200.0,3190.0,2760.0,16950.0,320.0,...,0.0,0.0,2780.0,4000.0,14210.0,39808.0,13950.0,38696.0,190.0,728.0
9212,36,NY,99999,3,11330.0,6670.0,2900.0,1470.0,10930.0,200.0,...,0.0,0.0,2630.0,5774.0,8200.0,22246.0,7930.0,20761.0,210.0,847.0
9213,36,NY,99999,4,7560.0,3990.0,2770.0,680.0,7280.0,170.0,...,0.0,0.0,2320.0,6591.0,4710.0,13680.0,4470.0,12070.0,170.0,891.0
9214,36,NY,99999,5,13140.0,5650.0,6530.0,680.0,12670.0,270.0,...,90.0,25.0,5350.0,26414.0,7290.0,30923.0,6500.0,24102.0,750.0,4723.0
9215,36,NY,99999,6,11710.0,3800.0,7430.0,330.0,11380.0,270.0,...,8470.0,308736.0,5770.0,226905.0,5500.0,589978.0,2950.0,70447.0,2680.0,477511.0


In [5]:
ny_df.shape

(9216, 167)

In [11]:

ny_df.to_csv('./data/ny_irs_data.csv', sep=',', index=False)

<h2>Get NY CSV DATA</h2>

In [2]:
df = pd.read_csv('./data/ny_irs_data.csv', header=0, dtype={'zipcode': str})

In [3]:
df.head()

,STATEFIPS,STATE,zipcode,agi_stub,N1,mars1,MARS2,MARS4,ELF,CPREP,...,N85300,A85300,N11901,A11901,N11900,A11900,N11902,A11902,N12000,A12000
0,36,NY,00000,1,2691220.0,2004510.0,257970.0,375610.0,2526010.0,84410.0,...,0.0,0.0,333540.0,282862.0,2120650.0,5354373.0,2106320.0,5299094.0,15870.0,39469.0
1,36,NY,00000,2,2324490.0,1357130.0,393190.0,515930.0,2242480.0,43080.0,...,0.0,0.0,421060.0,619115.0,1891640.0,5311653.0,1877170.0,5245116.0,19460.0,58182.0
2,36,NY,00000,3,1424670.0,779250.0,348060.0,247450.0,1371930.0,25640.0,...,0.0,0.0,358700.0,731938.0,1065060.0,2850715.0,1049050.0,2766543.0,21670.0,78743.0
3,36,NY,00000,4,908050.0,408800.0,342880.0,121210.0,871550.0,13540.0,...,0.0,0.0,285340.0,785409.0,617770.0,1944049.0,602360.0,1854707.0,16390.0,75357.0
4,36,NY,00000,5,1493450.0,449260.0,876140.0,120400.0,1437630.0,24540.0,...,9530.0,2643.0,623100.0,2893434.0,869030.0,3678321.0,822860.0,3277872.0,57010.0,402093.0


In [4]:
df.loc[0][0:10]

STATEFIPS           36
STATE               NY
zipcode          00000
agi_stub             1
N1           2691220.0
mars1        2004510.0
MARS2         257970.0
MARS4         375610.0
ELF          2526010.0
CPREP          84410.0
Name: 0, dtype: object

<h3>Write to PSQL database</h3>

In [25]:
# PSQL db connection using psycopg2
conn = psycopg2.connect(dbname='jhu', 
                            user='jhu', 
                            password='jhu123', 
                            host='postgres', 
                            port='5432')
c = conn.cursor()



# Create new schema in jhu database
c.execute('CREATE SCHEMA IF NOT EXISTS final')

# clearing table to start fresh
c.execute('DROP TABLE IF EXISTS final.ny_irs_data')
# Create table if doesn't exist
c.execute('''CREATE TABLE IF NOT EXISTS final.ny_irs_data 
            (STATEFIPS VARCHAR(50), STATE TEXT, zipcode TEXT, agi_stub NUMERIC, N1 NUMERIC, mars1 NUMERIC, MARS2 NUMERIC, MARS4 NUMERIC, ELF NUMERIC, CPREP NUMERIC,
            PREP NUMERIC, DIR_DEP NUMERIC, VRTCRIND NUMERIC, N2 NUMERIC, TOTAL_VITA NUMERIC, VITA NUMERIC, TCE NUMERIC, VITA_EIC NUMERIC, RAC NUMERIC, ELDERLY NUMERIC,
            A00100 NUMERIC, N02650 NUMERIC, A02650 NUMERIC, N00200 NUMERIC, A00200 NUMERIC, N00300 NUMERIC, A00300 NUMERIC, N00600 NUMERIC, A00600 NUMERIC, N00650 NUMERIC,
            A00650 NUMERIC, N00700 NUMERIC, A00700 NUMERIC, N00900 NUMERIC, A00900 NUMERIC, N01000 NUMERIC, A01000 NUMERIC, N01400 NUMERIC, A01400 NUMERIC, N01700 NUMERIC,
            A01700 NUMERIC, SCHF NUMERIC, N02300 NUMERIC, A02300 NUMERIC, N02500 NUMERIC, A02500 NUMERIC, N26270 NUMERIC, A26270 NUMERIC, N02900 NUMERIC, A02900 NUMERIC,
            N03220 NUMERIC, A03220 NUMERIC, N03300 NUMERIC, A03300 NUMERIC, N03270 NUMERIC, A03270 NUMERIC, N03150 NUMERIC, A03150 NUMERIC, N03210 NUMERIC, A03210 NUMERIC,
            N02910 NUMERIC, A02910 NUMERIC, N04450 NUMERIC, A04450 NUMERIC, N04100 NUMERIC, A04100 NUMERIC, N04200 NUMERIC, A04200 NUMERIC, N04470 NUMERIC, A04470 NUMERIC,
            A00101 NUMERIC, N17000 NUMERIC, A17000 NUMERIC, N18425 NUMERIC, A18425 NUMERIC, N18450 NUMERIC, A18450 NUMERIC, N18500 NUMERIC, A18500 NUMERIC, N18800 NUMERIC,
            A18800 NUMERIC, N18460 NUMERIC, A18460 NUMERIC, N18300 NUMERIC, A18300 NUMERIC, N19300 NUMERIC, A19300 NUMERIC, N19500 NUMERIC, A19500 NUMERIC, N19530 NUMERIC,
            A19530 NUMERIC, N19550 NUMERIC, A19550 NUMERIC, N19570 NUMERIC, A19570 NUMERIC, N19700 NUMERIC, A19700 NUMERIC, N20950 NUMERIC, A20950 NUMERIC, N04475 NUMERIC,
            A04475 NUMERIC, N04800 NUMERIC, A04800 NUMERIC, N05800 NUMERIC, A05800 NUMERIC, N09600 NUMERIC, A09600 NUMERIC, N05780 NUMERIC, A05780 NUMERIC, N07100 NUMERIC,
            A07100 NUMERIC, N07300 NUMERIC, A07300 NUMERIC, N07180 NUMERIC, A07180 NUMERIC, N07230 NUMERIC, A07230 NUMERIC, N07240 NUMERIC, A07240 NUMERIC, N07225 NUMERIC,
            A07225 NUMERIC, N07260 NUMERIC, A07260 NUMERIC, N09400 NUMERIC, A09400 NUMERIC, N85770 NUMERIC, A85770 NUMERIC, N85775 NUMERIC, A85775 NUMERIC, N10600 NUMERIC,
            A10600 NUMERIC, N59660 NUMERIC, A59660 NUMERIC, N59720 NUMERIC, A59720 NUMERIC, N11070 NUMERIC, A11070 NUMERIC, N10960 NUMERIC, A10960 NUMERIC, N11560 NUMERIC,
            A11560 NUMERIC, N11450 NUMERIC, A11450 NUMERIC, N11520 NUMERIC, A11520 NUMERIC, N11530 NUMERIC, A11530 NUMERIC, N10970 NUMERIC, A10970 NUMERIC, N10971 NUMERIC,
            A10971 NUMERIC, N06500 NUMERIC, A06500 NUMERIC, N10300 NUMERIC, A10300 NUMERIC, N85530 NUMERIC, A85530 NUMERIC, N85300 NUMERIC, A85300 NUMERIC, N11901 NUMERIC,
            A11901 NUMERIC, N11900 NUMERIC, A11900 NUMERIC, N11902 NUMERIC, A11902 NUMERIC, N12000 NUMERIC, A12000 NUMERIC)''')

# commit changes
conn.commit()

In [ ]:
# to copy data into psql, run following commands:
# psql -h localhost -p 5432 -U jhu -d jhu (or other database)
# COPY schema.table_name FROM '/home/jhu/...(filepath to csv)' DELIMITER',' CSV HEADER NULL 'NA"

In [20]:
conn.close()

In [22]:
conn = psycopg2.connect(dbname='jhu', 
                            user='jhu', 
                            password='jhu123', 
                            host='postgres', 
                            port='5432')
c = conn.cursor()

<h3>AFTER DATA COPIED</h3>

In [23]:
c.execute('''ALTER TABLE final.ny_irs_data ADD COLUMN id SERIAL PRIMARY KEY''')
conn.commit()

In [ ]:
# remove scientific notation from column data